# Regresion Logística a dataset pacientes de Covid-19 México(Spanish)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# librerias para datos y numeros
import numpy as np
import pandas as pd
# librerias para graficar
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# el modelo
from sklearn.linear_model import LogisticRegression
# metricas para evaluar al modelo
from sklearn import metrics
# importamos el modulo para cross validation
from sklearn.model_selection import cross_val_score
# el separador de datos en entrenamiento y testeo
from sklearn.model_selection import train_test_split

path = '/kaggle/input/cov19-open-data-mexico/data.csv'

# Preparación de datos
El dataset incluye muchas cosas, pero pasa que para este estudio solo se requieren unos cuantos de estos. `Este proceso lo que hará es extraer esos datos relevantes` para entrenar al modelo. El dataset se encuentra en `./../datasets/Mexico/datos_abiertos.csv`, junto a este hay varios xlsx con una descripción del contenido del dataset y marcadas las columnas que contienen informacion relevante, asi como el target.

In [ ]:
# primero traemos el dataset
dataset = pd.read_csv(path, encoding='unicode_escape')
print('Shape:', dataset.shape)
dataset.head()

In [ ]:
# vemos los datos que existen en la columna de CLASIFICACION_FINAL
dataset.CLASIFICACION_FINAL.value_counts().plot(kind='bar')

## Vemos que no tenemos valores nulos

In [ ]:
dataset.isnull().any()

## Selección de pacientes positivos
Debemos de seleccionar los pacientes que dieron positivo en covid para a ellos aplicarles el estudio, así que eliminaré los registros cuyo resulatdo fue negativo o pendiente, es decir, `que en la variable "CLASIFICACION_FINAL" tengan un 1 y de no ser así el registro será eliminado`. Se seleccionó esa variable puesto que en el PDF de la información del dataset aclara que esa será la variable en la cual se basará el conteo de casos COVID-19, pero `unicamente las que tienen un valor 1`.

In [ ]:
# definimos los valores que sean referentes a casos negativos
negative_values = (4,5,6,7,97)

# eliminar las filas de casos negativos
for val in negative_values:
  dataset = dataset.drop(dataset[dataset['CLASIFICACION_FINAL'] == val].index, axis=0)

# vemos cuanto queda del dataset
dataset.shape

Entonces ya nos quedamos unicamente con los registros de todos los casos positivos acumulados. El primer valor del shape indica la cantidad de registros que tenemos.

In [ ]:
dataset.head()

## Selección de features
Los `features son como las columnas o informacion relevante contenida en el dataset`. En este caso serían las enfermedades crónicas de los pacientes, si fuman, y su edad. así que en esta parte seleccionamos las partes o columnas que nos importan del dataset, como los son las enfermedades, si fue intubado, si presentó neumonía, etc.
Nota: también seleccionaremos el target ('FECHA_DEF') para que 

In [ ]:
# definimos las columnas que nos interesan para el cuestionamiento de las enfermedades cronicas
enfermedades = ['DIABETES','EPOC','ASMA','INMUSUPR','HIPERTENSION','CARDIOVASCULAR','OBESIDAD',
                'RENAL_CRONICA','TABAQUISMO','INTUBADO','NEUMONIA','EDAD','FECHA_DEF']

# seleccionamos las columnas de enfermedades
df = dataset[enfermedades]
df.head()

## Etiquetas

Eh aquí las etiquetas:
- Sexo: 1-Mujer, 2-Hombre
- Enfermedades: 1-Si se padece, 2-No se padece, 99-desconocido. `Al parecer en todas`.
- Fecha de defuncion: 9999-99-99 - no ha muerto, `una fecha diferente` - si murió.

En el caso de los resultados cambió el formato, el caso es que solo nos importan los resultados de 1,2,3. Y no los 4,5,6,7. Dichos datos vienen en la carpeta de *datos_abiertos_info*.

La celda de pruebas será la de abajo. PD: Los valores de 98, indican un 'No Especificado' así como los 99.

In [ ]:
# Graficamos cuantos valores diferentes hay en la columna de diabetes y cuantos hay de cada uno
df.DIABETES.value_counts().plot(kind='bar')

## Eliminación de valores no especificados
Como vemos en la grafica anterior la mayoría son 2 (que no tienen diabetes), hay pocos 1 (que si tienen diabets) y por último un 98 (no específifcado). Esa última columna es la que hay que eliminar ya que genera problemas para el eprendizaje del modelo y se encuentra en todas las columnas de enfermedades. Así que `hay que eliminar las columnas con valores de 98`.

In [ ]:
# ahora debemos retirar los valores de 98 o desconocido de todas nuestras columnas
# para esto iteramos las columnas, ya que este valor se da en todas
for column in df.columns:
    # eliminamos las filas que tengan un valor igual a 98
    df = df.drop(df[df[column]==98].index, axis=0)
    df = df.drop(df[df[column]==99].index, axis=0) 

In [ ]:
# Ya retiardos esos valores comprobamos y vemos que ya no esta ese 98
df.DIABETES.value_counts().plot(kind='bar')

## Convertir a valores booleanos
Los datos previamente vienen en un formato algo extraño, este es el caso de las enfermedades y el tabaquismo ya que `vienen entre 2 y 1, pero preferentemente se desean en 1-0 (1-si, 0-no)`. Así que habra que convertir solo los 2 a 0 en esos casos. `El caso que da más problemas es el de Intubado ya que el formato es diferente a los demás`. Así como los demás ese se arragla también en la celda siguiente.

In [ ]:
# ya que tenemos los datos sin 98, vamos a cambiar los 2,1 a 1,0 segun las deducciones de datos

# antes limpiare las de INTUBADO ya que estas vienen algo diferentes
df['INTUBADO'] = df['INTUBADO'].replace({2: 1})
df['INTUBADO'] = df['INTUBADO'].replace({97: 0})

# arreglamos las columnas de las enfermedades cronicas
# iteramos las columnas
for column in df.columns:
    # remplazamos los valores que queremos por los que queremos, de 2 a 0
    df[column] = df[column].replace({2: 0})

df.head()

## El target
El target es el resultado. Lo que el modelo tratará de `aprender será la relación numérica entre los fetures y el target`, es decir, la relación numérica entre las enfermedades, edad y otros datos del paciente y si este murío o no.
Así que el target será un array de 1 y 0 basado en si hubo fecha de defunción del paciente, `tal que sea un 1 si murío y 0 si no`.
En la úlitma linea de la celda siguiente se elimina la FECHA_DEF del dataframe que será usado para el modelo.

In [ ]:
# ahora el target, que seria si hubo defuncion, es en esta parte, ya que eliminamos los datos 98
target = []
# iteramos la columna de fecha de defuncion
for value in df.FECHA_DEF.values:
    if value == '9999-99-99': # no hubo defuncion
        target.append(0)
    else: # hubo defuncion
        target.append(1)
        
# por ultimo los dejamos en un array de numpy
target = np.array(target)
# tal que tendremos un 1 si murio o un 0 si no
print(target[:30], target.shape)

# Como previamente a df le agregamos 'FECHA_DEF' ahora se lo debemos de quitar        
df = df.drop(['FECHA_DEF'], axis=1)

In [ ]:
# esta es la cantidad de muertos, puede que sea algo diferente a la oficial, esto se debe a que
# previamente se eliminaron algunas columnas con datos inexactos.
print('Muertos:', list(target).count(1))

Corroboramos que coincidan los tamaños del dataset (X) que se usará pera entrenar al modelo y el tamaño del target(Y) que tambien será usado para eso.

En caso de no cincidir estos tamaños pasará que el programa dará error desde la parte de separación de datos.

In [ ]:
print('Muertos: ', list(target).count(1))
print('Casos target: ', list(target.shape)[0])
print('Casos Dataset: ', list(df.shape)[0])

## Visualización final de los datos
En esta celda veremos graficas de pastel que muestran que fraccion de los pacientes cuentan con algún factor. Por ejemplo: la gráfica de tabaquismo, muestra en azul la gente que no fuma y en naranja la gente que si fuma.

In [ ]:
# Ya tenemos todo, ahora una visualizacion de esto
# definimos las cuantas columnas y filas
rows = 4
cols = 3
# creamos el plot
fig, axs = plt.subplots(rows, cols)
# definimos los tiulos de los plots, que seran las columnas, pero sin la edad
titles = list(df.columns[:-1])
# y le agregamos a las columnas las defunciones
titles.append('DEFUNCIONES')
# contador auxiliar para indice de titulos
count = 0

# ietramos las filas
for i in np.arange(rows):
    # iteramos las columnas
    for j in np.arange(cols):
        # este if es por si los datos estan contenidos en df
        if count < 8:
            # definimos los valores para la grafica
            vals = [df[titles[count]].value_counts()[0], df[titles[count]].value_counts()[1]]
            axs[i, j].pie(vals, labels=['No','Si'])
        # en este pasa que los datos estan en target
        else:
            vals = [list(target).count(0), list(target).count(1)]
            axs[i, j].pie(vals, labels=['No','Si'])
        # seteamos el titulo con count y lo aumentamos
        axs[i, j].set_title(titles[count])
        count +=1

# Separacion de datos en Train y Test
Ya que tenemos el df que vendría siendo X y el target que sería Y, Vamos a separarlos en datos de entrenamiento para el modelo y en datos de testeo, los de testeo se encontraran en menor cantidad.

In [ ]:
# Separamos los datos en entrenamiento y testeo
X_train, X_test, Y_train, Y_test = train_test_split(df, target, test_size=0.3, random_state=0)

# para convencion definimos X, Y
X = df
Y = target

# El modelo
Ya que tenemos los datos preparados, Procedemos a crear el modelo.
Para este punto los datos son:
- X: dataframe con columnas de enfermedades y edad -> `df`
- Y: array de numpy con 1 o 0 indicando defunciones -> `target`

In [ ]:
# creamos el modelo
model = LogisticRegression(max_iter=len(X_train))
# entrenamos al modelo
model.fit(X_train, Y_train)
# y hacemos la prediccion con datos de testeo
Y_pred = model.predict(X_test)

# Evaluaciones del modelo

## Matriz de confusión
Una manera de graficar las predicciones y ver en cierto modo la asertividad de el modelo. Se lee de la siguiente manera: `Lo mas importante es la diagonal` ya que los numeros de esta representan los datos predichos correctamente, los que estan fuera de esta seran falsos negativos o falsos positivos, es decir, casos predichos erroneamente.

In [ ]:
# sacamos la matriz de confusion
cnf_matrix = metrics.confusion_matrix(Y_test, Y_pred)

In [ ]:
# creamos varios plots
fig, ax = plt.subplots()
# creamos etiquetas para las metricas de la grafica
class_names = [0,1]
tick_marks = np.arange(len(class_names))
# y se las ponemos
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# creamos los colores con ayuda de seaborn
sns.heatmap(pd.DataFrame(cnf_matrix), annot = True, cmap="Reds_r", fmt="g")

# poner la etiqueta para las x arriba
ax.xaxis.set_label_position("top")
# 
plt.tight_layout()
# definimos el titulo de la grafica
plt.title("Matriz de confusion", y=1.1)
# pnemos las etiquetas para x,y
plt.ylabel("Etiqueta Actual")
plt.xlabel("Etiqueta de prediccion")
# y mostramos
plt.show()

## Presición del modelo
Imprimimos la precision del modelo, esta se calcula comparando las predicciones del modelo hechas a partir de las X de testeo (Y_pred) con las Y de testeo (Y_test), Siendo esta como una especie de examen al aprendizaje del modelo.

In [ ]:
print('Exactitud de', round(metrics.accuracy_score(Y_test, Y_pred)*100, 2),'%')

## Cross Validation
Vamos a usar la Cross validation para `obtener un score mas robusto para el modelo`, es decir, evaluar la precision del modelo mediante un testeo mas riguroso.

Lo que hace la cross validation es `separar el set de datos en n partes, una de ellas será usada para test y las demas para train`. Este proceso se realizará n veces. El score se calculará con el promedio de los scores de casa iteración. Es computacionalmente costoso. Se dice que cuando n (n partes) es mayor a 5, los scores de los testeos no tendran variaciones significativas, asi que `por convención usamos 5`.

In [ ]:
# calculamos los scores dichos previamente
scores = cross_val_score(LogisticRegression(max_iter=len(Y)), X, Y, cv=5)
# y vemos los resultados
print(scores)

In [ ]:
print(f'Cross Validation score: {scores.mean()}')

# Predicciones
Esta es la parte que se usa para jugar con los valores y obtener respuestas a preguntas mediante predicciones hechas con estos valores.

## Predicción individual
Aquí veras una lista de todos los rubros tomados en cuenta por el modelo de ML, todos ellos tienen unos dos puntos y un valor entre corchetes, este indica con `un 1 si se tiene el problema y un 0 si no`, todos excepto la edad, esa es numerica. En la siguiente celda podrás `variar estos valores y ver cual es la probabilidad de defunción`.

In [ ]:
# ajuste de datos del paciente para predecir
paciente = pd.DataFrame({
    'DIABETES': [0],
    'EPOC': [0],
    'ASMA': [0],
    'INMUSUPR': [0],
    'HIPERTENSION': [0],
    'CARDIOVASCULAR': [0],
    'OBESIDAD': [0],
    'RENAL_CRONICA': [0],
    'TABAQUISMO': [0],
    'INTUBADO': [1],
    'NEUMONIA': [1],
    'EDAD': [80]
})
# predicciones del modelo
print(f'Probabilidad de defunsion = {round(model.predict_proba(paciente)[0,1]*100, 2)}%')
print(f'Probabilidad de salvarse = {round(model.predict_proba(paciente)[0,0]*100, 2)}%')

## Proyecciones con el modelo
En la celda de abajo se podrá modificar las facciones de los pacientes, y posteriormente graficarlas conforme el paso de la edad de el paciente, generando así una `curva de probabilidades de mortalidad` elaborada mediante las predicciones del modelo a los diferentes pacientes definidos.

In [ ]:
# vamos a ver como es que progresa la probabilidad de deceso segun la edad y otros factores
# listas para las predicciones de cada paciente
preds_1 = []
preds_2 = []
preds_3 = []
preds_4 = []
preds_5 = []
# iteramos los años de edad
for edad in np.linspace(1,100,100):
    # Aqui podrás ajustar los rasgos o factores de cada paciente como el ejemplo anterior
    paciente_1 = pd.DataFrame({
        'DIABETES': [0],
        'EPOC': [0],
        'ASMA': [0],
        'INMUSUPR': [0],
        'HIPERTENSION': [0],
        'CARDIOVASCULAR': [0],
        'OBESIDAD': [0],
        'RENAL_CRONICA': [0],
        'TABAQUISMO': [0],
        'INTUBADO': [1],
        'NEUMONIA': [1],
        'EDAD': [edad]
    })
    paciente_2 = pd.DataFrame({
        'DIABETES': [1],
        'EPOC': [0],
        'ASMA': [0],
        'INMUSUPR': [0],
        'HIPERTENSION': [0],
        'CARDIOVASCULAR': [0],
        'OBESIDAD': [0],
        'RENAL_CRONICA': [0],
        'TABAQUISMO': [0],
        'INTUBADO': [1],
        'NEUMONIA': [1],
        'EDAD': [edad]
    })
    paciente_3 = pd.DataFrame({
        'DIABETES': [0],
        'EPOC': [1],
        'ASMA': [0],
        'INMUSUPR': [0],
        'HIPERTENSION': [0],
        'CARDIOVASCULAR': [0],
        'OBESIDAD': [0],
        'RENAL_CRONICA': [0],
        'TABAQUISMO': [0],
        'INTUBADO': [1],
        'NEUMONIA': [1],
        'EDAD': [edad]
    })
    paciente_4 = pd.DataFrame({
        'DIABETES': [0],
        'EPOC': [0],
        'ASMA': [0],
        'INMUSUPR': [0],
        'HIPERTENSION': [0],
        'CARDIOVASCULAR': [0],
        'OBESIDAD': [1],
        'RENAL_CRONICA': [0],
        'TABAQUISMO': [0],
        'INTUBADO': [1],
        'NEUMONIA': [1],
        'EDAD': [edad]
    })
    paciente_5 = pd.DataFrame({
        'DIABETES': [0],
        'EPOC': [0],
        'ASMA': [0],
        'INMUSUPR': [0],
        'HIPERTENSION': [0],
        'CARDIOVASCULAR': [0],
        'OBESIDAD': [0],
        'RENAL_CRONICA': [1],
        'TABAQUISMO': [0],
        'INTUBADO': [1],
        'NEUMONIA': [1],
        'EDAD': [edad]
    })
    # agregamos las predicciones de cada paciente
    preds_1.append(model.predict_proba(paciente_1)[0,1])
    preds_2.append(model.predict_proba(paciente_2)[0,1])
    preds_3.append(model.predict_proba(paciente_3)[0,1])
    preds_4.append(model.predict_proba(paciente_4)[0,1])
    preds_5.append(model.predict_proba(paciente_5)[0,1])
    
# ploteamos las predicciones de distintos colores y les agregamos etiquetas
plt.plot(np.linspace(1,100,100), preds_1, ls='-' , color='red', label='Healhy')
plt.plot(np.linspace(1,100,100), preds_2, ls='dashed' , color='blue', label='Diabetes')
plt.plot(np.linspace(1,100,100), preds_3, ls='dashed' , color='green', label='EPOC')
plt.plot(np.linspace(1,100,100), preds_4, ls='dashdot', color='black', label='Obesity')
plt.plot(np.linspace(1,100,100), preds_5, color='orange', label='Chronic Renal')
# agregamos titulos y etiquetas al plot
plt.title('The patient has been intubated and has Neumony', )
plt.xlabel('Age', )
plt.ylabel('Probability', )
plt.xticks()
plt.yticks()
plt.legend()
plt.show()

## Detalles y algunas limitaciones.
Es posible que con `algunas combinaciones de problemas de salud o factores la curva de probabilidades no sea la esperada`, esto se puede deber a que hay pocos pacientes que cumplan con esa combinacion de problemas de salud o enfermedades, ocasionando que la predicción del modelo sea algo dudosa. Este problema parece presentarse en aquellos features o indicadores en los cuales los valores positivos son escasos. Esto último podemos `consultarlo en la sección de 'Visualización final de datos'`. Cabe recalcar que esta afirmación es `solo válida en algunos casos muy marcados` como por ejemplo el feature de Asma.

# Cuestionamientos:

### ¿Qué tan probable es la defunción según la edad, enfermedades cardiovasculares y diabetes?
En base a las graficas creadas a partir de predicciones del modelo vemos que `la enfermedad con mayor riesgo de deceso es la diabetes seguda por la obesidad`. Sin embargo vemos que estas curvas crecen cuando el paciente presenta neumonía, pero esta curvatura crece agresívamente cuando los pacientes han sido intubados y diagnosticados con neumonía.


En conclusión a esta pregunta, las respuestas vienen gráficamente en los plots de arriba. `En conclusión vemos que las enfermedades que más influyen son la diabetes y la obesidad`. Estas de entre las seleccionadas por el cuestionamiento (Obesidad, diabetes y cardiovasculares).

### ¿Qué tanto influye el tabaquismo y la edad en una posible defunción?
La mortalidad de las personas que fuman debería ser relativamente alta por sentido común, más sin embargo parace que esto no es así. Al parecer la gente que tiene el hábito de fumar no se encuentra entre los rangos más altos de mortalidad.

Un detalle a recalcar es que la gente con `asma y que fuma presenta una menor mortalidad a la que solo fuma`, esto se puede deber a que no hay tantos casos de gente con asma y que fume en el dataset, cosa por la que el modelo asume que tienen menos mortalidad, `en otras palabras, los detecta con menos mortalidad por ser sumamente raros esos casos`.

### ¿Qué enfermedades crónicas contribuyen más a la probabilidad de defunción?
Con las predicciones del modelo de la amplia variedad de enfermedades y padecimientos que nos ofrece el dataset podemos graficar y ver múltuples curvas de mortalidad. En las siguientes gráficas vemos como en las enfermedades varía la mortalidad y también como esta crece bastante cuando el paciente fué intubado y diagnosticado con neumonía.

En respuesta al análisis y vista de las gráficas vemos en estas que las `mortalidades mas áltas son: Enfermedad Renal Crónica, Diabetes, Inmunosupresión, Hipertención, Obesidad y Enfermedades Cardiovasculares`. Aún así siendo un paciente sin alguna de estas enfermedades la probabilidad al ser intubado sigue siendo una considerablemente elavada. 

Por último podemos ver las probabilidades de `mortalidad del Asma y la EPOC no son tan elevadas como las demás`, cosa que parece algo raro, debido a que el COVID-19 ataca a los pulmones principalmente. Estos resultados se `pueden deber a que el dataset con el que el modelo fue entrenado no habían tantos pacientes con estos padecimientos respiratorios` por lo que el modelo no pudo aprender del todo bien esos features o datos.


## Conclusión
Actualmente el COVID-19 está azotando agresivamente a nuestro país ya que `en base a las cifras registradas a nivel nacional la tasa de contagios sigue aumentando y la de mortalidad está en aproximadamente 10%, en otras palabras, muere 1 de cada 10 pacientes` aproximadamente. De aquí surge el quiestionamiento de que pacientes son los más propensos a fallecer.

En base a las respuestas a los cuestionamientos anteriores y a las diversas proyecciones hechas por el modelo podemos ver como es que las `personas con enfermedades cardiovasculares, obesidad, diabetes, renal crónica, e inmunosupresión son las que presentan mayores probabilidades de fallecer`. Por otro lado cuando el paciente ya ha sido `diagnosticado con neumonía y posteriormente intubado a un respirador artificial las probabilidades de defuncion crecen brúscamente`. Esto se puede apreciar en las proyecciones mostradas en el primer y tercer cuestionamiento.

En conclusión son aquellas personas las que padecen de `enfermedades cardiovasculares, diabetes, y enfermedades renales` aquellas que son las que presentan una mayor probabilidad de deceso por COVID-19 y por ende `aquellas que deben y debemos de tratar de cuidar más` para tratar de disminuir el crecimiento de la curva de mortalidad por COVID-19 en México.